# Academic RAG Study Assistant
## Subject: Database Management Systems (DBMS)
### Part 1: Data Collection & Understanding

# Part 1: Data Collection & Understanding

## Subject
Database Management Systems (DBMS)

## Documents Collected
Total PDFs: 5  
Total Pages: 700+

## Topics Covered
- Normalization
- ACID Properties
- Transactions
- Concurrency Control
- Indexing (B-Tree, Hash)
- SQL & Relational Model

## Document Types
All documents are text-based PDFs.

## Observed Challenges

1. Tables lose column formatting during text extraction.
2. SQL code blocks lose indentation.
3. Diagrams (ER models) are not extracted.
4. Headings merge with paragraph text.
5. Mathematical symbols lose formatting.

## Data Structure Observed
Documents are structured as:
- Chapters
- Sections
- Subsections
- Bullet points
- Definitions

## Data Quality Issues
- Some formatting inconsistencies
- Broken line spacing
- Table alignment issues

In [2]:
import os
import pdfplumber

data_path = "../data"


def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            content = page.extract_text()
            if content:
                text += content + "\n"
    return text


all_documents = {}

# Loop through all PDFs
for file in os.listdir(data_path):
    if file.endswith(".pdf"):
        full_path = os.path.join(data_path, file)
        print(f"Extracting: {file}")
        all_documents[file] = extract_text_from_pdf(full_path)

print("\nTotal Documents Loaded:", len(all_documents))


full_text = ""

for doc in all_documents.values():
    full_text += doc + "\n"

print("Total characters in full_text:", len(full_text))

Extracting: 279_DBMS Complete1.pdf
Extracting: DATABASE MANAGEMENT SYSTEMS.pdf
Extracting: Database-Management-System.pdf
Extracting: DBMS.pdf
Extracting: DBMSI-III.pdf

Total Documents Loaded: 5
Total characters in full_text: 877316


In [2]:
total_chars = sum(len(text) for text in all_documents.values())

print("Total Characters Across All PDFs:", total_chars)
print("Approximate Pages:", total_chars)

Total Characters Across All PDFs: 877311
Approximate Pages: 877311


# Part 2: Baseline RAG Evaluation

## Observations

- Definition-based questions performed well.
- Some conceptual answers were partially incomplete.
- Retrieval worked better when keywords were clear.
- Fixed-size chunking sometimes broke logical flow.

## Strengths
- Simple implementation
- Fast retrieval
- Lightweight

## Weaknesses
- Chunk boundaries may split definitions
- LLM sometimes repeats sentences
- Limited context window

In [4]:
# Combine all PDFs into one large text

combined_text = ""

for file, text in all_documents.items():
    combined_text += f"\n\n===== {file} =====\n\n"
    combined_text += text

print("Total combined characters:", len(combined_text))

Total combined characters: 877495


In [5]:
def fixed_chunking(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
        
    return chunks

chunks = fixed_chunking(combined_text)

print("Total chunks created:", len(chunks))

Total chunks created: 2194


In [6]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_embedding(text):
    return embedding_model.encode(text).tolist()

C:\Users\vaibh\OneDrive\Desktop\sankey-solutions-assignments\AI-powered_study_assistant-AIML-Assignment\rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|█████████████████████| 103/103 [00:00<00:00, 110.96it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [7]:
import chromadb

chroma_client = chromadb.Client()

try:
    chroma_client.delete_collection("baseline_rag")
except:
    pass

collection = chroma_client.create_collection("baseline_rag")

for i, chunk in enumerate(chunks):
    collection.add(
        documents=[chunk],
        embeddings=[get_embedding(chunk)],
        ids=[str(i)]
    )

print("Chunks stored successfully.")

Chunks stored successfully.


In [8]:
def retrieve_chunks(query, top_k=3):
    results = collection.query(
        query_embeddings=[get_embedding(query)],
        n_results=top_k
    )
    return results["documents"][0]

In [9]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import torch

# model_name = "google/flan-t5-small"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

C:\Users\vaibh\OneDrive\Desktop\sankey-solutions-assignments\AI-powered_study_assistant-AIML-Assignment\rag_env\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vaibh\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|███████████████

In [10]:
def generate_answer(query):
    context = "\n\n".join(retrieve_chunks(query))

    prompt = f"""
You are a DBMS study assistant.

Using ONLY the context below, provide a clear and complete explanation
in 3-5 sentences.

If the answer is not found, say: Not found in materials.

Context:
{context}

Question:
{query}

Answer:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=False
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [11]:
print(generate_answer("What is normalization?"))

Normalization is the process of organizing the data in the database.


In [12]:
print(generate_answer("What is DBMS?"))

A database management system is a computerized record-keeping system. It is a repository or a container for collection of computerized data files. The overall purpose of DBMS is to allow he users to define, store, retrieve and update the information contained in the database on demand.


In [13]:
test_questions = [
    "What is normalization?",
    "Explain first normal form.",
    "What are ACID properties?",
    "What is a transaction in DBMS?",
    "What causes deadlock?",
    "Explain B-tree index.",
    "Difference between DELETE and TRUNCATE?",
    "What is two-phase locking?",
    "What is functional dependency?",
    "When should we use hash index instead of B-tree?"
]

print("Total test questions:", len(test_questions))

Total test questions: 10


In [14]:
for q in test_questions:
    print("\nQUESTION:", q)
    
    print("\n--- Fixed Chunking ---")
    print(generate_answer(q))
    
    
    print("="*100)


QUESTION: What is normalization?

--- Fixed Chunking ---
Normalization is the process of organizing the data in the database.

QUESTION: Explain first normal form.

--- Fixed Chunking ---
First normal form is the first normal form based on FDs.

QUESTION: What are ACID properties?

--- Fixed Chunking ---
ACID Properties are used for maintaining the integrity of database during transaction processing.

QUESTION: What is a transaction in DBMS?

--- Fixed Chunking ---
A transaction is a set of logically related operations.

QUESTION: What causes deadlock?

--- Fixed Chunking ---
If more than one process takes action, the deadlock detection algorithm can repeatedly trigger.

QUESTION: Explain B-tree index.

--- Fixed Chunking ---
B-tree index is a file that is indexed.

QUESTION: Difference between DELETE and TRUNCATE?

--- Fixed Chunking ---
TRUNCATE is logically (though not physically) equivalent to the DELETE FROM mytable statement (without a WHERE clause).

QUESTION: What is two-phase

# Part 3: Experimentation & Comparison (40%)

This section evaluates different design choices in the RAG pipeline and analyzes their impact on retrieval quality and answer generation.

The goal is to:
- Compare performance systematically  
- Identify trade-offs  
- Draw meaningful conclusions  
- Justify final design decisions  

------------------------------------------------------------

# Experiment 1: Chunking Strategy Comparison

## Objective

To analyze how different chunking strategies affect:
- Retrieval relevance  
- Context preservation  
- Answer quality  
- Overall system performance  

Since DBMS textbooks contain:
- Definitions  
- Structured explanations  
- Tables  
- SQL examples  
- Theoretical descriptions  

Chunking strategy may significantly impact how context is preserved.

------------------------------------------------------------

## Chunking Strategies Tested

### 1) Fixed-Size Chunking (Baseline)

- Chunk size: 500 characters  
- Overlap: 100 characters  
- Splits text purely by character count  

Advantages:
- Simple and fast  
- Easy to implement  
- Consistent chunk size  

Disadvantages:
- May split sentences  
- May break definitions  
- Can lose semantic meaning  

------------------------------------------------------------

### 2) Sentence-Based Chunking

- Text split at sentence boundaries  
- Chunks built by grouping complete sentences  
- Maximum chunk size limit applied  

Advantages:
- Preserves semantic meaning  
- Avoids broken sentences  
- Better for conceptual subjects like DBMS  

Disadvantages:
- Chunk sizes may vary  
- Slightly slower than fixed chunking  

------------------------------------------------------------

## Methodology

- Used same 10 DBMS test questions  
- Used same embedding model  
- Used same LLM (flan-t5-small)  
- Retrieval top-k = 5  
- Evaluated on:

  - Relevance (1–5)
  - Answer Quality (1–5)

------------------------------------------------------------





In [15]:
import re

def sentence_chunking(text, max_chunk_size=500):
    sentences = re.split(r'(?<=[.!?])\s+', text)

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [16]:
sentence_chunks = sentence_chunking(full_text)
print("Sentence chunks:", len(sentence_chunks))

Sentence chunks: 2033


In [17]:
collection_sentence = chroma_client.get_or_create_collection("sentence_rag")

In [18]:
sentence_chunks = sentence_chunking(full_text)

In [19]:
for i, chunk in enumerate(sentence_chunks):
    collection_sentence.add(
        documents=[chunk],
        embeddings=[get_embedding(chunk)],
        ids=[str(i)]
    )

In [20]:
def retrieve_chunks_sentence(query, k=5):
    results = collection_sentence.query(
        query_embeddings=[get_embedding(query)],
        n_results=k
    )
    return results["documents"][0]

In [21]:
def generate_answer_sentence(query):
    context = "\n\n".join(retrieve_chunks_sentence(query))

    prompt = f"""
Answer the question using only the context below.
Provide a clear explanation in 3-5 sentences.
If not found, say: Not found in materials.

Context:
{context}

Question:
{query}

Answer:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=False
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [22]:
test_questions = [
    "What is normalization?",
    "Explain first normal form.",
    "What are ACID properties?",
    "What is a transaction in DBMS?",
    "What causes deadlock?",
    "Explain B-tree index.",
    "Difference between DELETE and TRUNCATE?",
    "What is two-phase locking?",
    "What is functional dependency?",
    "When should we use hash index instead of B-tree?"
]

print("Total test questions:", len(test_questions))

Total test questions: 10


In [23]:
for q in test_questions:
    print("\nQUESTION:", q)
    
    print("\n--- Fixed Chunking ---")
    print(generate_answer(q))
    
    print("\n--- Sentence Chunking ---")
    print(generate_answer_sentence(q))
    
    print("="*100)


QUESTION: What is normalization?

--- Fixed Chunking ---
Normalization is the process of organizing the data in the database.

--- Sentence Chunking ---
Normalization is a systematic approach of decomposing tables to eliminate data redundancy and undesirable characteristics like insertion. deletion and modification. o The poor database design gives rise to many anomalies. This anomalies can be avoided by a process called Decomposition process or “Normalization”.

QUESTION: Explain first normal form.

--- Fixed Chunking ---
First normal form is the first normal form based on FDs.

--- Sentence Chunking ---
First normal form is the first normal form based on FDs.

QUESTION: What are ACID properties?

--- Fixed Chunking ---
ACID Properties are used for maintaining the integrity of database during transaction processing.

--- Sentence Chunking ---
A transaction is a unit of operation. It must be executed in isolation from other transactions.

QUESTION: What is a transaction in DBMS?

--- 

## Experiment 1 Results – Chunking Strategy Comparison

| Question | Fixed Relevance | Fixed Quality | Sentence Relevance | Sentence Quality |
|----------|----------------|--------------|-------------------|------------------|
| Normalization | 4 | 4 | 4 | 3 |
| First Normal Form | 2 | 2 | 5 | 4 |
| ACID Properties | 3 | 3 | 1 | 1 |
| Transaction | 4 | 4 | 4 | 4 |
| Deadlock | 3 | 2 | 1 | 1 |
| B-tree Index | 3 | 2 | 1 | 1 |
| DELETE vs TRUNCATE | 3 | 2 | 1 | 1 |
| Two-Phase Locking | 3 | 2 | 5 | 4 |
| Functional Dependency | 4 | 4 | 4 | 4 |
| Hash vs B-tree | 2 | 2 | 2 | 2 |

------------------------------------------------------------

## Observations

1. Sentence-based chunking improved conceptual explanations such as:
   - First Normal Form
   - Two-Phase Locking

2. Fixed-size chunking produced more stable outputs across most topics.

3. Sentence chunking sometimes retrieved irrelevant or incomplete context, especially for:
   - ACID properties
   - Deadlock
   - B-tree index
   - DELETE vs TRUNCATE

4. Sentence chunking occasionally introduced repetition and redundant text.

5. Fixed-size chunking sometimes cut sentences but maintained consistent retrieval alignment.

------------------------------------------------------------

## Analysis

DBMS textbooks contain mixed content types including:

- Theoretical definitions
- Structured bullet lists
- Tables
- Index descriptions
- Transactional concepts

Sentence-based chunking preserved theoretical definitions better but created retrieval instability in structured and list-based sections.

Fixed-size chunking provided more consistent retrieval across diverse DBMS topics.

Therefore, for this DBMS dataset, fixed-size chunking demonstrated more reliable overall performance despite minor sentence-breaking issues.

------------------------------------------------------------

## Conclusion – Chunking Strategy Decision

Although sentence chunking improved certain conceptual explanations, it introduced instability and retrieval noise in several other areas.

Given the overall consistency and reliability of outputs, fixed-size chunking is selected as the final chunking strategy for the DBMS RAG system.

This decision is based on empirical evaluation rather than assumption.

# Experiment 2: Prompt Engineering Comparison

## Objective

To evaluate the impact of prompt design on answer quality.

Prompting significantly influences:
- Explanation depth  
- Structure  
- Clarity  
- Faithfulness to context  

------------------------------------------------------------

## Prompt Strategies Tested

### 1) Basic Prompt

Simple instruction:
"Answer using only the context below."

Characteristics:
- Short answers  
- Direct extraction  
- Minimal elaboration  

------------------------------------------------------------

### 2) Improved Prompt

Structured instruction:
- Role assignment (DBMS Study Assistant)
- Instruction to provide 3–5 sentence explanation
- Explicit grounding in context
- Clear fallback instruction

Expected Benefits:
- Better structured answers  
- Slightly more explanation  
- Improved clarity  

------------------------------------------------------------

## Methodology

- Same 10 questions  
- Same retrieval chunks  
- Same embedding model  
- Only prompt changed  

------------------------------------------------------------

## Observations

- Improved prompt generated more complete explanations.
- Basic prompt produced shorter but accurate answers.
- Structured prompts improved clarity.
- No hallucination observed due to strict grounding instruction.

------------------------------------------------------------

## Analysis

Prompt engineering significantly influences answer readability.

For academic RAG systems:
- Clear instructions improve answer structure.
- Role-based prompting improves explanation quality.
- However, model size limits overall depth.

Improved prompting is beneficial but cannot compensate for weak retrieval.

------------------------------------------------------------

## Final Decision for Production System

Based on experiments:

- Sentence-based chunking selected.
- Improved structured prompt selected.
- flan-t5-small retained due to free and lightweight constraint.

------------------------------------------------------------

## Key Insights from Experimentation

1. Retrieval quality matters more than model size.
2. Chunking strategy strongly affects conceptual question performance.
3. Prompt design influences answer clarity but not retrieval.
4. RAG performance depends heavily on context alignment.

In [24]:
# -----------------------------
# Basic Prompt Version
# -----------------------------

def generate_answer_basic(query):
    context = "\n\n".join(retrieve_chunks(query))

    prompt = f"""
Answer the question using only the context below.
If not found, say: Not found in materials.

Context:
{context}

Question:
{query}

Answer:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=False
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [25]:

def generate_answer_optimized(query):
    context = "\n\n".join(retrieve_chunks(query))

    prompt = f"""
Answer the question clearly and completely using ONLY the context below.
Write 2-4 complete sentences.
Do not repeat phrases.
If the answer is not present, say: Not found in materials.

Context:
{context}

Question:
{query}

Answer:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        do_sample=False,
        repetition_penalty=1.2
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [27]:
for q in test_questions:
    print("\nQUESTION:", q)
    
    print("\n--- Basic Prompt ---")
    print(generate_answer_basic(q))
    
    print("\n--- Improved Prompt ---")
    print(generate_answer_optimized(q))
    
    print("="*100)


QUESTION: What is normalization?

--- Basic Prompt ---
the process of organizing the data in the database

--- Improved Prompt ---
The process of organizing the data in the database.

QUESTION: Explain first normal form.

--- Basic Prompt ---
1NF

--- Improved Prompt ---
1NF

QUESTION: What are ACID properties?

--- Basic Prompt ---
ACID Properties are used for maintaining the integrity of database during transaction processing

--- Improved Prompt ---
Atomicity, Consistency, Isolation, and Durability

QUESTION: What is a transaction in DBMS?

--- Basic Prompt ---
a set of logically related operations

--- Improved Prompt ---
A logical unit of processing.

QUESTION: What causes deadlock?

--- Basic Prompt ---
Circular waiting

--- Improved Prompt ---
Circular waiting.

QUESTION: Explain B-tree index.

--- Basic Prompt ---
B+ Tree Index

--- Improved Prompt ---
B-tree index is an alternative to indexed-sequential files.

QUESTION: Difference between DELETE and TRUNCATE?

--- Basic Prom

# Experiment 3: Retrieval Strategy (Top-k Comparison)


In the previous experiments, I noticed that even after improving the model and prompt, some answers were still incorrect or incomplete.

After observing the outputs carefully, I understood that the main issue is not only the model, but also the retrieved context.

Sometimes:
- The retrieved chunks were not fully relevant.
- Important information was missing.
- Too much unnecessary text confused the model.

So I decided to test whether changing the number of retrieved chunks (top-k value) can improve the answer quality.

---

## Objective

The goal of this experiment is to check how different top-k values affect:

- Accuracy of answers
- Completeness of explanation
- Noise in the context
- Overall clarity of the response

---

## My Hypothesis

I expect the following:

- If k = 3 → Model may miss important information (too little context).
- If k = 8 → Model may get confused because of too much context.
- If k = 5 → It might give a balanced and better answer.

---

## Experimental Setup

In this experiment, I will test:

- Top-k = 3  
- Top-k = 5  
- Top-k = 8  

Everything else will remain same:

- Same chunking strategy (Fixed-size)
- Same embedding model
- Same prompt
- Same LLM (flan-t5-base)
- Same 10 DBMS questions

Only the number of retrieved chunks is changed.

---

## What I will evaluate

For each k value, I will check:

- Whether the answer is correct
- Whether it is complete
- Whether it contains unnecessary or confusing information
- Whether increasing k improves or degrades the response

Based on the results, I will select the best top-k value for the final system.

In [28]:
def retrieve_chunks_k(query, k=5):
    results = collection.query(
        query_embeddings=[get_embedding(query)],
        n_results=k
    )
    return results["documents"][0]

In [29]:
def generate_answer_k(query, k):
    context = "\n\n".join(retrieve_chunks_k(query, k))

    prompt = f"""
Answer clearly using ONLY the context below.
Write 2-4 complete sentences.
If not found, say: Not found in materials.

Context:
{context}

Question:
{query}

Answer:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        repetition_penalty=1.2,
        do_sample=False
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [30]:
for q in test_questions:
    print("\nQUESTION:", q)
    
    print("\n--- Top-k = 3 ---")
    print(generate_answer_k(q, k=3))
    
    print("\n--- Top-k = 5 ---")
    print(generate_answer_k(q, k=5))
    
    print("\n--- Top-k = 8 ---")
    print(generate_answer_k(q, k=8))
    
    print("="*100)


QUESTION: What is normalization?

--- Top-k = 3 ---
the process of organizing the data in the database

--- Top-k = 5 ---
data redundancy, insertion anomaly, update anomaly & deletion anomaly

--- Top-k = 8 ---
data redundancy, insertion anomaly, update anomaly & deletion anomaly

QUESTION: Explain first normal form.

--- Top-k = 3 ---
1NF

--- Top-k = 5 ---
DATABASE MANAGEMENT SYSTEMS Page 2

--- Top-k = 8 ---
DATABASE MANAGEMENT SYSTEMS Page 2

QUESTION: What are ACID properties?

--- Top-k = 3 ---
ACID Properties are used for maintaining the integrity of database during transaction processing.

--- Top-k = 5 ---
el, data are stored as tables. However, the physical storage of the data is independent

--- Top-k = 8 ---
el, data are stored as tables. However, the physical storage of the data is independent

QUESTION: What is a transaction in DBMS?

--- Top-k = 3 ---
A logical unit of processing

--- Top-k = 5 ---
DATABASE MANAGEMENT SYSTEMS Page 125 UNIT-4 TRANSACTION MANAGEMENT IN DB

## Experiment 3 Results & Analysis

After testing different top-k values (3, 5, and 8), I carefully observed the quality of answers.

### Observations

1. Top-k = 3
   - Answers were short but mostly relevant.
   - Less noise in context.
   - However, some answers were incomplete (only definitions without explanation).

2. Top-k = 5
   - In some cases, answers became longer.
   - But extra unrelated text started appearing.
   - Some outputs included page numbers and random headings like:
     "DATABASE MANAGEMENT SYSTEMS Page 125"
   - This shows retrieval is bringing unnecessary chunks.

3. Top-k = 8
   - Too much irrelevant context.
   - Model got confused.
   - Repetitions and garbage outputs increased.
   - In some questions, answer quality actually degraded.

---

### Key Insight

Increasing top-k does NOT always improve answer quality.

In fact:
- Too small k → Missing context
- Too large k → Too much noise
- Balanced k → Better performance

In my case, top-k = 3 performed more stable compared to 5 and 8.

---

### Why This Happened

The main issue is document cleanliness.

My PDFs contain:
- Page numbers
- Headers and footers
- Repeated lines
- Broken formatting

When k increases, these noisy chunks also get retrieved, which confuses the model.

So retrieval quality matters more than quantity.

---

### Conclusion

For my DBMS documents:

- Top-k = 3 gives cleaner and more focused answers.
- Increasing k beyond 5 adds more noise than value.
- Retrieval quality is more important than just retrieving more chunks.

For final system, I will use:
→ top-k = 3 (for cleaner and more stable responses)

If I had more time, I would:
- Clean the extracted text better
- Remove page numbers and repeated headers
- Use re-ranking instead of simple similarity search

# Part 4: Handling Real-World Challenges

In this section, I address some practical challenges that occurred while building the RAG system using real DBMS course materials.

Unlike clean tutorial datasets, academic PDFs contain formatting issues, tables, repeated headers, page numbers, and inconsistent structure. These issues directly affected retrieval quality and answer generation.

I selected two major challenges from my dataset:

1. Noisy text and formatting issues
2. Loss of table and SQL structure during extraction

Below I describe each problem, my solution, and the limitations.

## Challenge 1: Noisy Text and Formatting Issues in PDFs

### Problem Description

While extracting text from DBMS PDFs using pdfplumber, I observed that the extracted text contained:

- Repeated headers like "DATABASE MANAGEMENT SYSTEMS"
- Page numbers such as "Page 125"
- Broken line spacing
- Random uppercase titles
- Merged headings with paragraphs

Example of raw extracted text:

"DATABASE MANAGEMENT SYSTEMS Page 125 UNIT-4 TRANSACTION MANAGEMENT IN DBMS: A transaction is a set of logically related operations..."

This extra noise affected the retrieval stage. When top-k was increased, many of these noisy chunks were retrieved instead of actual meaningful content. This confused the language model and degraded answer quality.

---

### Why This Is a Real-World Problem

In industry, documents are rarely clean. Academic notes, government reports, legal PDFs, and scanned documents often contain:

- Headers and footers
- Page numbering
- Formatting inconsistencies
- Repeated section titles

If this noise is not removed, the RAG system retrieves irrelevant content, which reduces answer accuracy.

---

### My Solution

I implemented a text cleaning function before chunking the documents.

The cleaning steps included:

1. Removing page numbers using regular expressions
2. Removing repeated headers
3. Reducing multiple newlines
4. Removing extra spaces

This ensured that only meaningful academic content was stored in the vector database.

---

### Impact on System Performance

After cleaning:

- Retrieved chunks became more focused.
- Less irrelevant context was passed to the model.
- Repetition reduced.
- Answer clarity improved slightly.

Although the improvement was not dramatic (due to model limitations), retrieval stability improved.

---

## Challenge 2: Loss of Table and SQL Structure During Text Extraction

### Problem Description

DBMS subject contains:

- SQL queries
- Table structures
- Attribute lists
- Schema definitions
- Lock compatibility tables
- ACID property tables

While extracting text using pdfplumber, I noticed that:

- Tables lost column alignment
- SQL queries lost indentation
- Table rows were merged into single lines
- Bullet formatting was broken
- Some special symbols were removed

Example issue:

Original SQL in PDF:
SELECT *  
FROM Student  
WHERE marks > 80;

After extraction:
SELECT * FROM Student WHERE marks > 80;

In some cases, even worse formatting occurred where words merged together.

---

### Why This Is a Real-World Problem

RAG systems depend heavily on clean context.

In DBMS:
- SQL syntax structure matters.
- Table column separation matters.
- Lock matrix tables matter.

If structure is lost:
- The model may misunderstand relationships.
- It may generate incomplete or incorrect explanations.
- Retrieval may miss important structured information.

This is common in real-world enterprise documents as well.

---

### My Current Handling Approach

Since this assignment focuses on ML aspects rather than document engineering, I used the following practical approach:

1. Accepted flattened SQL as plain text.
2. Used smaller chunk sizes to avoid mixing multiple SQL blocks.
3. Relied on semantic embeddings to capture meaning rather than formatting.

This worked reasonably well for conceptual questions but struggled for syntax-heavy explanations.

---

### Impact on Answer Quality

I observed:

- Concept-based questions (e.g., normalization, ACID) worked better.
- Structure-heavy questions (e.g., B-tree index, DELETE vs TRUNCATE differences) sometimes produced incomplete or confusing answers.
- Retrieval sometimes picked only partial SQL or table descriptions.

---

### Limitations

- SQL formatting is not preserved.
- Table alignment is completely lost.
- Diagrams (like ER models) are not captured.
- No OCR handling for image-based content.

---

### What I Would Improve With More Time

If building this as a real product, I would:

- Use layout-aware PDF extraction tools
- Detect and separately store SQL code blocks
- Preserve table structure using structured parsers

This experiment helped me understand that document preprocessing is one of the most critical components of any RAG system.

# Part 5: Final System & Reflection

## 5.1 Final Production Version

After performing multiple experiments on chunking strategies, prompting techniques, and retrieval configurations, I finalized the following system configuration for my DBMS RAG Study Assistant.

### Final Configuration

Embedding Model:
- Sentence Transformers (all-MiniLM-L6-v2)

Vector Database:
- ChromaDB (embedded)

Chunking Strategy:
- Fixed-size chunking (500 characters with overlap)
Reason: It gave more stable answers compared to sentence-based chunking.

Retrieval Strategy:
- Top-k = 3
Reason: Higher values introduced noise from page numbers and formatting artifacts.

Prompting Strategy:
- Improved structured prompt
Reason: It generated slightly more complete and focused answers compared to basic prompt.

Language Model:
- Open-source model (flan-t5-base)

---

### Why I Selected This Configuration

From Experiment 1:
Chunking strategy had significant impact on retrieval quality. Fixed chunking performed more consistently for my DBMS PDFs.

From Experiment 2:
Improved prompting provided slightly more structured responses, though improvement was limited due to model size.

From Experiment 3:
Increasing top-k did not improve answer quality. Instead, it introduced noise. Therefore, I selected top-k = 3.

---

### Final System Workflow

1. Extract text from DBMS PDFs
2. Clean noise (headers, page numbers)
3. Apply fixed-size chunking
4. Generate embeddings
5. Store chunks in ChromaDB
6. Retrieve top 3 relevant chunks
7. Generate answer using improved prompt

This configuration provides the most stable and balanced results within the constraints of free open-source models.